In [1]:
# https://github.com/zbanga/FXCM_currency/blob/master/Automated%20Algo.ipynb

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import fxcmpy
import time
from talib import abstract
import datetime as dt
import datetime
import numpy as np
import pandas as pd
import math
import time
import cufflinks as cf  # Cufflinks
cf.set_config_file(offline=True)  # set the plotting mode to offline

import plotly.graph_objs as go

/home/mik/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
api = fxcmpy.fxcmpy(config_file='FXCM.cfg')

In [4]:
# More documentation http://fxcmpy.tpq.io/
# Calling API to get data 
# change start and stop times
# Instrument must be one of ('EUR/USD', 'USD/JPY', 'GBP/USD', 'USD/CHF', '
# EUR/CHF', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'EUR/GBP', 'EUR/JPY', 'GBP/JPY',
#'AUD/JPY', 'USD/CNH', 'FRA40', 'GER30', 'UK100', 'US30', 'USDOLLAR', 'XAU/USD', 'XAG/USD').

candles = api.get_candles('EUR/USD', period='m5',
                         start=dt.datetime(2018, 2, 7),
                          stop=dt.datetime(2018, 2, 22))

In [5]:
#Putting data into a dataframe with a midclose 
data = pd.DataFrame(candles[['askclose', 'bidclose']].mean(axis=1),
                    columns=['midclose'])

In [6]:
data['spread']= candles['askclose'] - candles['bidclose']

In [7]:
data['spread'].head()

date
2018-02-07 00:00:00    0.00021
2018-02-07 00:05:00    0.00022
2018-02-07 00:10:00    0.00023
2018-02-07 00:15:00    0.00025
2018-02-07 00:20:00    0.00023
Name: spread, dtype: float64

In [8]:
###OOS
candles_oos = api.get_candles('EUR/USD', period='m5',
                         start=dt.datetime(2018, 6, 23),
                          stop=dt.datetime(2018, 8, 8))
#Putting data into a dataframe with a midclose 
data_oos = pd.DataFrame(candles_oos[['askclose', 'bidclose']].mean(axis=1),
                    columns=['midclose'])
data_oos.info()

data_oos.iplot()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9338 entries, 2018-06-24 19:00:00 to 2018-08-08 00:00:00
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   midclose  9338 non-null   float64
dtypes: float64(1)
memory usage: 145.9 KB


In [9]:
data_oos.describe()

,midclose
count,9338.000000
mean,1.166481
std,0.005222
min,1.152925
25%,1.164016
50%,1.166635
75%,1.170340
max,1.178925


<h2>Feature Preperation</h2>

In [10]:
### OOS
data_oos['returns'] = np.log(data_oos['midclose'] / data_oos['midclose'].shift(1))
lags = 5

data_oos['spread']= candles_oos['askclose'] - candles_oos['bidclose']

# Applying lag# Normalising Returns
# Applying lag
cols = []
for lag in range(1, lags + 1):
    col = 'lag_%s' % lag
    data_oos[col] = data_oos['returns'].shift(lag)
    cols.append(col)

data_oos['Ask_Dir'] = np.where(data_oos['returns'].shift(-1) > data_oos['returns'], 1, 0)
cols.append('Ask_Dir')

data_oos['SMA'] = abstract.SMA(data_oos['returns'], timeperiod=12)
cols.append('SMA')
data_oos['SMA_Dir'] = np.where(data_oos['SMA'].shift(-1) > data_oos.SMA, 1, 0)
cols.append('SMA_Dir')

data_oos['RSI'] = abstract.RSI(data_oos['returns'], timeperiod=12)
cols.append('RSI')
data_oos['RSI_Dir'] = np.where(data_oos['RSI'].shift(-1) > data_oos.RSI, 1, 0)
cols.append('RSI_Dir')

data_oos['fastk'], data_oos['fastd'] = abstract.STOCHRSI(data_oos['returns'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
cols.append('fastk')
cols.append('fastd')
data_oos['fastk_Dir'] = np.where(data_oos['fastk'].shift(-1) > data_oos.fastk, 1, 0)
cols.append('fastk_Dir')
data_oos['fastd_Dir'] = np.where(data_oos['fastd'].shift(-1) > data_oos.fastd, 1, 0)    
cols.append('fastd_Dir')    
    
  
    
# Adding features which such as rolling returns
col = 'momentum'
data_oos[col] = data_oos['returns'].rolling(5).mean().shift(1)
cols.append(col)#

cols

['lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'Ask_Dir',
 'SMA',
 'SMA_Dir',
 'RSI',
 'RSI_Dir',
 'fastk',
 'fastd',
 'fastk_Dir',
 'fastd_Dir',
 'momentum']

In [11]:
data_oos

,midclose,returns,spread,lag_1,lag_2,lag_3,lag_4,lag_5,Ask_Dir,SMA,SMA_Dir,RSI,RSI_Dir,fastk,fastd,fastk_Dir,fastd_Dir,momentum
date,,,,,,,,,,,,,,,,,,
2018-06-24 19:00:00,1.165745,NaN,0.00061,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0,NaN
2018-06-24 19:05:00,1.165750,0.000004,0.00072,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0,NaN
2018-06-24 19:10:00,1.165745,-0.000004,0.00061,0.000004,NaN,NaN,NaN,NaN,1,NaN,0,NaN,0,NaN,NaN,0,0,NaN
2018-06-24 19:15:00,1.165780,0.000030,0.00064,-0.000004,0.000004,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0,NaN
2018-06-24 19:20:00,1.165750,-0.000026,0.00070,0.000030,-0.000004,0.000004,NaN,NaN,1,NaN,0,NaN,0,NaN,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-07 23:40:00,1.160315,-0.000060,0.00023,0.000017,-0.000086,0.000203,0.000017,0.000082,0,0.000030,1,47.070119,0,9.790006,14.893506,0,0,0.000047
2018-08-07 23:45:00,1.160205,-0.000095,0.00023,-0.000060,0.000017,-0.000086,0.000203,0.000017,1,0.000030,0,45.916587,1,0.000000,14.893506,1,1,0.000018
2018-08-07 23:50:00,1.160315,0.000095,0.00025,-0.000095,-0.000060,0.000017,-0.000086,0.000203,0,0.000030,0,52.848768,0,100.000000,36.596669,0,1,-0.000004


In [12]:
data_oos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9338 entries, 2018-06-24 19:00:00 to 2018-08-08 00:00:00
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   midclose   9338 non-null   float64
 1   returns    9337 non-null   float64
 2   spread     9338 non-null   float64
 3   lag_1      9336 non-null   float64
 4   lag_2      9335 non-null   float64
 5   lag_3      9334 non-null   float64
 6   lag_4      9333 non-null   float64
 7   lag_5      9332 non-null   float64
 8   Ask_Dir    9338 non-null   int64  
 9   SMA        9326 non-null   float64
 10  SMA_Dir    9338 non-null   int64  
 11  RSI        9325 non-null   float64
 12  RSI_Dir    9338 non-null   int64  
 13  fastk      9317 non-null   float64
 14  fastd      9317 non-null   float64
 15  fastk_Dir  9338 non-null   int64  
 16  fastd_Dir  9338 non-null   int64  
 17  momentum   9332 non-null   float64
dtypes: float64(13), int64(5)
memory usage: 1.4 MB


In [13]:
data_oos.dropna(inplace=True)

In [14]:
data_oos

,midclose,returns,spread,lag_1,lag_2,lag_3,lag_4,lag_5,Ask_Dir,SMA,SMA_Dir,RSI,RSI_Dir,fastk,fastd,fastk_Dir,fastd_Dir,momentum
date,,,,,,,,,,,,,,,,,,
2018-06-24 20:55:00,1.166445,-0.000009,0.00071,0.000051,-0.000450,-0.000009,0.000274,0.000326,1,0.000057,0,49.233376,1,59.367825,39.881633,1,1,0.000039
2018-06-24 21:00:00,1.166560,0.000099,0.00044,-0.000009,0.000051,-0.000450,-0.000009,0.000274,0,0.000054,0,51.906618,0,100.000000,73.214966,0,1,-0.000028
2018-06-24 21:05:00,1.166460,-0.000086,0.00032,0.000099,-0.000009,0.000051,-0.000450,-0.000009,0,0.000050,0,47.239200,0,69.780318,76.382714,0,0,-0.000063
2018-06-24 21:10:00,1.166100,-0.000309,0.00032,-0.000086,0.000099,-0.000009,0.000051,-0.000450,1,0.000016,0,42.228482,1,0.000000,56.593439,1,0,-0.000079
2018-06-24 21:15:00,1.166140,0.000034,0.00032,-0.000309,-0.000086,0.000099,-0.000009,0.000051,1,0.000008,1,50.958402,1,89.020905,52.933741,1,1,-0.000051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-07 23:40:00,1.160315,-0.000060,0.00023,0.000017,-0.000086,0.000203,0.000017,0.000082,0,0.000030,1,47.070119,0,9.790006,14.893506,0,0,0.000047
2018-08-07 23:45:00,1.160205,-0.000095,0.00023,-0.000060,0.000017,-0.000086,0.000203,0.000017,1,0.000030,0,45.916587,1,0.000000,14.893506,1,1,0.000018
2018-08-07 23:50:00,1.160315,0.000095,0.00025,-0.000095,-0.000060,0.000017,-0.000086,0.000203,0,0.000030,0,52.848768,0,100.000000,36.596669,0,1,-0.000004


In [15]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
#model = LogisticRegression(C=100)
#model = GaussianNB()
model = MLPClassifier(hidden_layer_sizes=[100,50 ,100], max_iter=200)

In [17]:
#Fitting Model
%time model.fit(np.sign(data_oos[cols]), np.sign(data_oos['returns']))

CPU times: user 1min 11s, sys: 46.3 s, total: 1min 58s
Wall time: 34.8 s


MLPClassifier(hidden_layer_sizes=[100, 50, 100])

In [18]:
#Transforming model into 0 and 1 
pred = model.predict(np.sign(data_oos[cols]))
pred[:15]

array([-1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,
        1., -1.])

In [19]:
data_oos['position'] = pred

In [20]:
data_oos['strategy'] = data_oos['position'] * data_oos['returns']
#data_oos['strategy_tc'] = data_oos['position'] * data_oos['returns'] - data_oos['spread']

In [21]:
# unleveraged | no bid-ask spread or transaction costs | only in-sample
data_oos[['returns', 'strategy' ]].cumsum().apply(np.exp).iplot()

In [22]:
data_oos['position'].value_counts()

 1.0    5083
-1.0    4221
 0.0      13
Name: position, dtype: int64